In [1]:
# Эагрузка файла, предварительно скачанного с PubChem 
# по ссылке https://pubchem.ncbi.nlm.nih.gov/#query=PubChem%20Compound%20TOC%3A%20Toxicity
# Ссылка - это поиск по фразе 'PubChem Compound TOC: Toxicity', 
# выдает (предположительно) вещества, где в описании секция Toxicity не пустая
import pandas as pd
df = pd.read_csv(r'../data/raw/pubchem/PubChem_compound_cache_hcIgmJxu-dLO_HHl8504zfcrnUtTnzWLT64ux1S_PMZUpgA.csv')

In [2]:
# В моем файле дефект - лишний пробел в начале, поэтому 
# переименуем ' cid' в 'cid'
df = df.rename(columns={' cid': 'cid'})

In [104]:
# создадим пустой датафрейм для приема запрошенных данных
toxicity_data = pd.DataFrame(
    {
        "cid": pd.Series(dtype="int64"),
        "status_code": pd.Series(dtype="int16"),
        "toxicity_data_info": pd.Series(dtype="object"),
        "toxicity_data_value": pd.Series(dtype="object"),
    }
)

# заготовим пустые строки с номерами CID 
toxicity_data['cid'] = df.cid   

In [289]:
toxicity_data.dtypes, toxicity_data.shape

(cid                      int64
 status_code            float64
 toxicity_data_info      object
 toxicity_data_value     object
 dtype: object,
 (118039, 4))

In [281]:
import random
import time
import logging
# чтобы это сработало, предварительно выполнить в корне проекта
# $ pip install .
from datasets import pubchem as pc 

In [282]:
start = 0
stop = 118040 # это максимальный номер в скачанном файле


for index, row in toxicity_data.loc[start:stop].iterrows():
    # print(f"{index}: {row}")
    if row['status_code'] in [200, 404]:
        continue
    try:
        res = pc.get_toxicity_data(row['cid'])
        toxicity_data.at[index, 'status_code'] = int(res.status_code)
        toxicity_data.at[index, 'toxicity_data_info'] = res.json()
        toxicity_data.at[index, 'toxicity_data_value'] = [
            pc.get_toxicity_data_value(res.json()) if res.status_code == 200 else None
        ]
    except Exception as e:
        toxicity_data.at[index, 'status_code'] = 0
        toxicity_data.at[index, 'toxicity_data_info'] = None
        toxicity_data.at[index, 'toxicity_data_value'] = None
        logging.error(f"{index}/{stop}: cid={row['cid']}: {e}")
    print(f"{index}/{stop}: cid={row['cid']}: {res.status_code}")
    time.sleep(random.random()/3+0.2)